In [1]:
import time
import random
from ortools.linear_solver import pywraplp  # Google OR Tools - Linear Solver

# Problem Data
* Buyer wants to purchase items within the given budget.
    - B: maximum budget given to the buyer
* There are N distinct SKUs, each of which has its own purchase price (from supplier) and estimated PC1 contribution (per unit).
    - f[i]: PC1 per unit for the i-th item
    - p[i]: purchase price per unit for the i-th item

In [2]:
# SKUs
number_of_SKUs = 50
SKU = {}
for i in range(number_of_SKUs):  # Each item has its purchase price and PC1 contribution
    price = random.randint(20, 200)
    SKU[i] = {'p': price, 'f': price * random.randint(5, 20) / 100}

# Budget
B = 300000  # Maximum budget given to the buyer

# Variables
* Binary decision variable: x[i] ≥ 0 denotes the number of units to purchase
* We have N items, so we have N binary decision variables, x[1], x[2], ..., x[N].

In [3]:
solver = pywraplp.Solver.CreateSolver('SCIP')  # Solver object to use "SCIP" algorithm of the linear solver 

x = {}  # Binary decision variable, x[i] ≥ 0 which denotes the purshase quantity of the i-th SKU
for i in range(number_of_SKUs):
    x[i] = solver.IntVar(0, solver.Infinity(), f'x_{i}')

# Constraints
* The budget is limited. So, we need to make a decision which SKUs & how many to purchase within the budget (B).
* Anyway, we can model the constraint like this
    - p[1] * x[1] + p[2] * x[2] + ... + p[N] * x[N] <= B
    - sum{ p[i] * x[i] | i = 1, 2, ..., N } <= B, equivalently

In [4]:
# Constraint: total purchase price of the units to be purchased must be no greater than the buyer's given budget
solver.Add(
    sum(SKU[i]['p'] * x[i] for i in range(number_of_SKUs)) <= B
)

<ortools.linear_solver.pywraplp.Constraint; proxy of <Swig Object of type 'operations_research::MPConstraint *' at 0x7f23d8223960> >

# Objective Function
* Buyer wants to maximize the total PC1 of the purchased units.
* So, the objective function needs to take the total PC1 of the purchased items into account in its calculation.
    - Maximize ( f[1] * x[1] + f[2] * x[2] + ... + f[N] * x[N] )
    - equivalently, Maximize sum{ f[i] * x[i] | i = 1, 2, ..., N }

In [5]:
# Maximise the total PC1 of the units to purchase
solver.Maximize(
    sum(SKU[i]['f'] * x[i] for i in range(number_of_SKUs))
)

# Solve
### Linear Programming Model
- Maximize
    - sum{ f[i] x[i] | i = 1, 2, ..., N }
- Subject to
    - sum{ p[i] * x[i] | i = 1, 2, ..., N } ≤ B
    - x[i] ≥ 0 & integer for i = 1, 2, ..., N

In [6]:
# Solve the linear program
status = solver.Solve()

# Check the results
print('status =', status)  # 0=optimal, 1=feasible, 2=infeasible, 3=unbounded, 4=abnormal, 6=not solved
print('objective value =', solver.Objective().Value())
number_of_SKUs_purchased = 0
number_of_units_purchased = 0
total_budget_used = 0
for i in range(number_of_SKUs):
    if x[i].solution_value() > 0:
        total_budget_used += SKU[i]["p"] * x[i].solution_value()
        number_of_SKUs_purchased += 1
        number_of_units_purchased += x[i].solution_value()
print('total budget used =', total_budget_used)
print('number of SKUs purchased =', number_of_SKUs_purchased)
print('number of units purchased =', number_of_units_purchased)

status = 0
objective value = 60000.0
total budget used = 300000.0
number of SKUs purchased = 1
number of units purchased = 6000.0
